In [3]:
import pandas as pd
import numpy as np

In [5]:
df=pd.read_csv('monthly_macro_exchange_real_rates_2000_2024.csv')

In [7]:
df

,Date,Currency,ExchangeRate_to_USD,InterestRate,Inflation,GDPGrowth,Unemployment
0,2000-01-01,EUR,1.214488,0.472347,2.323844,3.956909,4.882923
1,2000-01-01,GBP,0.649122,1.315843,2.383717,3.359158,5.271280
2,2000-01-01,JPY,107.448961,0.006854,2.120981,2.926016,4.137541
3,2000-01-01,INR,44.709462,4.297434,2.157124,3.227593,4.293848
4,2000-01-01,USD,1.007328,1.454845,2.033764,3.072576,4.727809
...,...,...,...,...,...,...,...
1795,2024-12-01,GBP,0.746575,0.956879,2.793534,3.261252,4.888364
1796,2024-12-01,JPY,151.267261,0.269290,2.769792,3.321503,4.887324
1797,2024-12-01,INR,82.683332,4.259805,2.512080,3.530932,4.777527
1798,2024-12-01,USD,1.008005,1.573840,2.741827,2.529588,4.990341


In [9]:
df.shape

(1800, 7)

In [11]:
df.isnull().sum()

Date                   0
Currency               0
ExchangeRate_to_USD    0
InterestRate           0
Inflation              0
GDPGrowth              0
Unemployment           0
dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import joblib
import datetime

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Currency'] = le.fit_transform(df['Currency'])


In [14]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day']=df['Date'].dt.day

In [15]:
df.drop(["Date"], axis=1, inplace=True)

In [19]:
df

,Currency,ExchangeRate_to_USD,InterestRate,Inflation,GDPGrowth,Unemployment,Year,Month,Day
0,1,1.214488,0.472347,2.323844,3.956909,4.882923,2000,1,1
1,2,0.649122,1.315843,2.383717,3.359158,5.271280,2000,1,1
2,4,107.448961,0.006854,2.120981,2.926016,4.137541,2000,1,1
3,3,44.709462,4.297434,2.157124,3.227593,4.293848,2000,1,1
4,5,1.007328,1.454845,2.033764,3.072576,4.727809,2000,1,1
...,...,...,...,...,...,...,...,...,...
1795,2,0.746575,0.956879,2.793534,3.261252,4.888364,2024,12,1
1796,4,151.267261,0.269290,2.769792,3.321503,4.887324,2024,12,1
1797,3,82.683332,4.259805,2.512080,3.530932,4.777527,2024,12,1
1798,5,1.008005,1.573840,2.741827,2.529588,4.990341,2024,12,1


In [21]:
X = df.drop(['ExchangeRate_to_USD', 'Year', 'Month', 'Day'], axis=1).values
y = df['ExchangeRate_to_USD'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
models = {
    "Ridge": Ridge(),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, random_state=42)
}
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    results[name] = {"R²": r2, "MAE": mae, "RMSE": rmse}
    
    print(f"\n{name}:\n  R²: {r2:.4f}  MAE: {mae:.4f}  RMSE: {rmse:.4f}")
best_model_name = max(results, key=lambda name: results[name]["R²"])
best_model = models[best_model_name]
joblib.dump(best_model, f"best_model_{best_model_name}.pkl")

print(f"\n Best model: {best_model_name} saved as best_model_{best_model_name}.pkl")


Ridge:
  R²: 0.2196  MAE: 27.7480  RMSE: 36.2818

RandomForest:
  R²: 0.9813  MAE: 2.4252  RMSE: 5.6205

XGBoost:
  R²: 0.9765  MAE: 3.2771  RMSE: 6.3002

 Best model: RandomForest saved as best_model_RandomForest.pkl


In [22]:
for name, model in models.items():
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    results[name] = {
        'Train_R²': r2_score(y_train, y_train_pred),
        'Test_R²': r2_score(y_test, y_test_pred),
        'Train_MAE': mean_absolute_error(y_train, y_train_pred),
        'Test_MAE': mean_absolute_error(y_test, y_test_pred),
        'Train_RMSE': mean_squared_error(y_train, y_train_pred, squared=False),
        'Test_RMSE': mean_squared_error(y_test, y_test_pred, squared=False)
    }
results

{'Ridge': {'Train_R²': 0.15571443787672068,
  'Test_R²': 0.2196002154748069,
  'Train_MAE': 30.986723661065906,
  'Test_MAE': 27.748035579229867,
  'Train_RMSE': 39.17559930969654,
  'Test_RMSE': 36.28177350784957},
 'RandomForest': {'Train_R²': 0.9965721578979135,
  'Test_R²': 0.9812720850649393,
  'Train_MAE': 1.0863569080044158,
  'Test_MAE': 2.4252361413106804,
  'Train_RMSE': 2.496211456467567,
  'Test_RMSE': 5.620496109136833},
 'XGBoost': {'Train_R²': 0.9997198607703296,
  'Test_R²': 0.9764688247000195,
  'Train_MAE': 0.4135658065958486,
  'Test_MAE': 3.2770603982502537,
  'Train_RMSE': 0.7136050796102938,
  'Test_RMSE': 6.300162438013525}}